# Data Modeling
Zuerst vorbereitete Daten einlesen

In [ ]:
import pandas as pd;
import numpy as np;
from sklearn.model_selection import train_test_split
from sklearn import tree
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import export_graphviz

cleaned_data = pd.read_csv('cleaned_data.csv')

#### Location aufteilen in Dummy Varibalen 
Die Spalten AREA.NAME und RD werden durch dummy-0-1-Variablen abgebildet

In [ ]:
# Spalte LOCATION wird zu mehrere 0,1 Variablen um diese im Suchbaum abzubilden 

# One-Hot Encoding für das String-Feature "LOCATION"
area_names = list(map(lambda x: 'AREA.NAME_' + x, set(cleaned_data['AREA.NAME'].values)))
encoded_data = pd.get_dummies(cleaned_data, columns=['AREA.NAME'])
cleaned_data = pd.concat([encoded_data], axis=1)
# Anzeigen des transformierten DataFrames
print(area_names)
cleaned_data['RD'] = cleaned_data['RD'].astype(str)

# Definiere eine Funktion, um die letzten beiden Zeichen einer Zeichenkette zu entfernen
def remove_last_two_chars(text):
    return text[:-1]

# Wende die Funktion auf die Spalte "RD" an und speichere das Ergebnis in einer neuen Spalte "New_RD"
cleaned_data['New_RD'] = cleaned_data['RD'].apply(remove_last_two_chars)
cleaned_data['New_RD'] = cleaned_data['New_RD'].astype(int)
# Gib den aktualisierten DataFrame aus
cleaned_data


cleaned_data['New_RD'] = cleaned_data['New_RD'].astype(str)
rd_nr = list(map(lambda x: 'New_RD_' + x, set(cleaned_data['New_RD'].values)))
encoded_data = pd.get_dummies(cleaned_data, columns=['New_RD'])
cleaned_data = pd.concat([encoded_data], axis=1)
print(rd_nr)

# Gib den aktualisierten DataFrame aus
cleaned_data


In [ ]:
import pandas as pd

# Annahme: Du hast bereits einen DataFrame mit den Spalten "RD" und "LAT" erstellt.
# Du kannst deine Daten in den DataFrame "df" laden.

# Definiere die benutzerdefinierte Aggregationsfunktion, um den Mittelwert von "LAT" und den entsprechenden Wert von "RD" zu erhalten

# Konvertiere die Spalte "RD" in einen String-Datentyp
#cleaned_data['RD'] = cleaned_data['RD'].astype(str)

# Definiere eine Funktion, um die letzten beiden Zeichen einer Zeichenkette zu entfernen
#def remove_last_two_chars(text):
 #   return text[:-0]

# Wende die Funktion auf die Spalte "RD" an und speichere das Ergebnis in einer neuen Spalte "New_RD"
#cleaned_data['RD'] = cleaned_data['RD'].apply(remove_last_two_chars)


def custom_agg(series):
    return pd.Series({
        'LAT_mean': series['LAT'].mean(),
        'LONG_mean': series['LONG'].mean(),
        'RD_value': series.iloc[0]
    })

# Gruppiere den DataFrame nach der Spalte "RD" und wende die benutzerdefinierte Aggregationsfunktion an
result = cleaned_data.groupby('RD')['LAT', 'LONG'].apply(custom_agg).reset_index()

# Gib das Ergebnis aus
print(result)

In [ ]:
result.to_csv('ergebnis.csv', index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Lade die CSV-Datei in einen DataFrame
df = pd.read_csv('ergebnis.csv')

# Wähle die gewünschten Spalten aus
df_filtered1 = df[df['RD'].between(200, 299)]
df_filtered2 = df[df['RD'].between(300, 399)]
df_filtered3 = df[df['RD'].between(400, 499)]
df_filtered4 = df[df['RD'].between(500, 599)]

spalte1_1 = df_filtered1['LAT_mean']
spalte2_1 = df_filtered1['LONG_mean']
spalte1_2 = df_filtered2['LAT_mean']
spalte2_2 = df_filtered2['LONG_mean']
spalte1_3 = df_filtered3['LAT_mean']
spalte2_3 = df_filtered3['LONG_mean']
spalte1_4 = df_filtered4['LAT_mean']
spalte2_4 = df_filtered4['LONG_mean']
# Erstelle das Punktdiagramm

plt.scatter(spalte1_1, spalte2_1, c='red', label='Bereich 300-399')
plt.scatter(spalte1_2, spalte2_2, c='blue', label='Bereich 400-499')
plt.scatter(spalte1_3, spalte2_3, c='green', label='Bereich 500-599')
plt.scatter(spalte1_4, spalte2_4, c='grey', label='Bereich 600-699')

plt.ylabel('Längengrad (LONG)')
plt.xlabel('Breitengrad (LAT)')
plt.title('RD aufgeteilt nach AREA')
plt.legend()

# Setze die Range der x- und y-Achse auf feste Werte
x_range = (33.7, 34.4 )
y_range = (-118.7, -118.1)
plt.xlim(x_range)
plt.ylim(y_range)
plt.show()

modelle
xgboost
multilayerperception

## Getestete Modelle
Im folgenden werden einige Modelle mit den getesteten Parametern gezeigt. Dabei wurden natürlich nicht nur die gezeigten Features getestet. Insgesamt wurden für den DecisionTree ale Features geprüft und auch das Verhältnis zu den zu klassifizierenden Klassen `CRIME_VIOLENT`, `Crm.Cd` und `CRIME_CAT`. Mehr dazu im Projektbericht.

#### DecisionTree 1/4
- features = ['RD']
- target = 'CRIME_VIOLENT'

In [ ]:
predict_data = cleaned_data.copy()

# Schritt 2: Merkmale und Zielvariable definieren
features = ['RD', "LAT", "LONG"]
target = 'CRIME_VIOLENT'

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Modell erstellen und trainieren
model = DecisionTreeClassifier(max_depth = 8)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)

#### DecisionTree 2/4
- features = ['LAT','LONG', 'RD', 'TIME.OCC_hour_cos']
- target = 'CRIME_CAT'

performt schlechter als ohne  'TIME.OCC_hour_cos' - siehe DecisionTee 3/4

In [ ]:
predict_data = cleaned_data.copy()

# Schritt 2: Merkmale und Zielvariable definieren
features = ['LAT','LONG', 'RD', 'TIME.OCC_hour_cos']
target = 'CRIME_CAT'

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Modell erstellen und trainieren
model = DecisionTreeClassifier(max_depth = 80)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)

#### DecisionTree 3/4
- features = [ 'RD',  'LAT', 'LONG']
- target = 'CRIME_CAT'


In [ ]:
predict_data = cleaned_data.copy()

# Schritt 2: Merkmale und Zielvariable definieren
features = [ 'RD',  'LAT', 'LONG']
target = 'CRIME_CAT'


# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Modell erstellen und trainieren
model = DecisionTreeClassifier(max_depth = 30)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)

#### DecisionTree 4/4
- features = [ 'RD',  'LAT', 'LONG']
- target = 'Crm.Cd'


In [ ]:
predict_data = cleaned_data.copy()

# Schritt 2: Merkmale und Zielvariable definieren
features = ['RD', 'LAT', 'LONG']
target = 'Crm.Cd'


# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Modell erstellen und trainieren
model = DecisionTreeClassifier(max_depth = 40)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)

## Random Forest

#### RandomForest 1/4
- features = ['RD', 'LAT', 'LONG']
- target = 'CRIME_VIOLENT'

In [ ]:
predict_data = cleaned_data.copy()
from sklearn.ensemble import RandomForestClassifier
# Schritt 2: Merkmale und Zielvariable definieren
features = ['RD', 'LAT', 'LONG']
target = 'CRIME_VIOLENT'

# Remove all NaN values
predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Modell erstellen und trainieren

model = RandomForestClassifier(max_depth = 20, n_estimators=70)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells: " , accuracy)

#### RandomForest 2/4
- features = ['RD', 'LAT', 'LONG']
- target = 'CRIME_CAT'

Bisher bestes Ergebnis!

In [ ]:
predict_data = cleaned_data.copy()
from sklearn.ensemble import RandomForestClassifier
# Schritt 2: Merkmale und Zielvariable definieren
features = ['RD', 'LAT', 'LONG']
target = 'CRIME_CAT'

# Remove all NaN values
predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Modell erstellen und trainieren

model = RandomForestClassifier(max_depth = 24, n_estimators=64)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells: " , accuracy)
#Genauigkeit des Modells: deep: 24n_est: 64 0.3148376642071581

#### RandomForest 3/4
- features = ['RD', 'LAT', 'LONG']
- target = 'Crm.Cd'

In [ ]:
predict_data = cleaned_data.copy()
from sklearn.ensemble import RandomForestClassifier
# Schritt 2: Merkmale und Zielvariable definieren
features = ['RD', 'LAT','LONG']
target = 'Crm.Cd'


# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

model = RandomForestClassifier(max_depth = 40, n_estimators= 64)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)

#### RandomForest 4/4
- features = ['RD', 'LAT', 'LONG', 'TIME.OCC_hour_cos']
- target = 'Crm.Cd'

In [ ]:
predict_data = cleaned_data.copy()
from sklearn.ensemble import RandomForestClassifier
# Schritt 2: Merkmale und Zielvariable definieren
features = ['RD', 'LAT', 'LONG', 'TIME.OCC_hour_cos']
target = 'Crm.Cd'

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

model = RandomForestClassifier(max_depth = 80, n_estimators= 64)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)

## Sequentielles neuronales Netzwerk

####  Sequentielles neuronales Netzwerk 1/2
- Dense-Schicht
- target = 'CRIME_CAT'
- features = 'AREA', 'RD', 'LAT', 'LONG',  'TIME.OCC_hour_cos', 'DATE.OCC_day_cos', 'DATE.OCC_month_cos' AND ALL FEATURES
- viel Ausprobiert - wenig gutes

####  Sequentielles neuronales Netzwerk 2/2
- Dense-Schicht
- tensorflow Module müssen hierfür installiert werden

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Schritt 1: Daten laden und vorverarbeiten
predict_data = cleaned_data.copy()
target = 'Crm.Cd'

features = ['RD', 'LONG', 'LAT', 'TIME.OCC_hour_cos']
# Remove all NaN values
predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()


# Schritt 2: Daten in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)



# Schritt 5: Neuronales Netzwerk erstellen

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(16, activation='tanh', input_shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(16, activation='tanh'))


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train_encoded, epochs=2, batch_size=20, validation_split=0.2)

# Schritt 7: Vorhersagen treffen
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Schritt 8: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells: " + str(accuracy))

#höher als 42,98 ist nicht möglich. Habe activation = "tanh" und "relu" ausprobiert, 2 und 3 Layers und 32, 64 und 128 Neuronen. 1, 2, 10 Epochs und batchsize = 2, 10, 20 
#"mean_squared_error" funktioniert garnicht
#optimizer='adam' gut, optimizer='SGD' gut,  optimizer='Adagrad' gut
# okay. iwas ist komisch. mit 3 mal 2 Neuronen bekomme ich trd über 42%. Das ist komisch. - erklärbar: einteilung bei den Koordinaten macht, dass wir auf 42% kommen 



## Naive Bayes Klassifikatoren

#### Naive Bayes-Klassifikator (GaussianNB)
- Target: CRIME_CAT
- Features: ['UNIX.TIMESTAMP','AREA.NAME','LAT','LONG']
- viel Ausprobiert - wenig gutes

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.naive_bayes import GaussianNB


predict_data = cleaned_data.copy()
# Build a Gaussian Classifier
nb_class = GaussianNB()
# Prior-Verteilungen für jede Klasse definieren

features = ['UNIX.TIMESTAMP','LAT','LONG']
target = 'CRIME_CAT'
selection = features + [target]

predict_data = cleaned_data[selection].copy()
# Filtern von null (muss nur für eines gemacht werden, da Koordinaten nie alleine vorkommen)
predict_data = predict_data[predict_data['LONG'].notnull()]


X_train, X_test, y_train, y_test = train_test_split(predict_data.drop(target, axis=1), predict_data[target], test_size=0.2, random_state=42)
nb_class.fit(X_train,y_train)

print("Model Accuracy:")
nb_class.score(X_test,y_test)
#ohne Priors: 42,811
#probiert Priors anzupassen. Komme auf Maximal 21% WK. lohnt nicht
#priors = [0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0565] - 8,9%

#Auch weitere Anpassungen brachten keine Verbesserung:
#print("Accuracy (Standardized):", accuracy_standardized)
#print("Accuracy (Normalized):", accuracy_normalized)

#### Naive Bayes-Klassifikator (CategoricalNB)
- Target: CRIME_CAT
- Features: ['TIME.OCC_hour','DATE.OCC_weekday','DATE.OCC_day','DATE.OCC_month','DATE.OCC_year','RD','LOCATION.street']
- deutlich besser als Gauß, kann auch Straßen verwenden

In [ ]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.calibration import LabelEncoder

# Build a Categorical Classifier
nb_class = CategoricalNB()

features = ['TIME.OCC_hour','DATE.OCC_weekday','DATE.OCC_day','DATE.OCC_month','DATE.OCC_year','RD','LOCATION.street']
target = 'CRIME_CAT'
selection = features + [target]
predict_data_encoded = cleaned_data[selection].copy()
predict_data_encoded['LOCATION.street']= LabelEncoder().fit_transform(predict_data_encoded['LOCATION.street'])

X_train, X_test, y_train, y_test = train_test_split(predict_data_encoded.drop(target, axis=1), predict_data_encoded[target], test_size=0.2, random_state=42)
nb_class.fit(X_train,y_train)

print(f'Model Accuracy: {nb_class.score(X_test,y_test)}')

### LogisticRegression
- dauert viel zu lange
- nicht viel besser

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

predict_data = cleaned_data.copy()

# Schritt 2: Merkmale und Zielvariable definieren
target = 'CRIME_CAT'
#column_to_exclude = ['Date.Rptd', 'DR.NO', 'DATE.OCC', 'TIME.OCC', 'AREA', 'AREA.NAME', 'RD', 'Crm.Cd', 'CrmCd.Desc', 'Status', 'Status.Desc', 'LOCATION', 'Cross.Street', 'LAT', 'LONG', 'UNIX.TIMESTAMP', 'TIME.OCC_hour', 'DATE.OCC_day', 'DATE.OCC_weekday', 'DATE.OCC_month', 'DATE.OCC_year', 'CRIME_VIOLENT', 'CRIME_CAT', 'Cos_Uhrzeit', 'TIME.OCC_hour_cos', 'DATE.OCC_day_cos', 'DATE.OCC_month_cos']
column_to_exclude = ['Date.Rptd', 'DR.NO', 'DATE.OCC', 'TIME.OCC', 'Crm.Cd', 'CrmCd.Desc', 'Status', 'Status.Desc', 'LOCATION.street', 
                     'Cross.Street.street', 'LOCATION.house_number', 'Cross.Street.house_number','UNIX.TIMESTAMP', 'TIME.OCC_hour', 
                     'DATE.OCC_day', 'DATE.OCC_weekday', 'DATE.OCC_month', 'DATE.OCC_year', 'CRIME_VIOLENT', 'CRIME_CAT']
#Enthalten: 'AREA', 'RD', 'LAT', 'LONG',  'TIME.OCC_hour_cos', 'DATE.OCC_day_cos', 'DATE.OCC_month_cos'
features = predict_data.drop([target] + column_to_exclude, axis=1).columns

# Remove all NaN values
predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Logistische Regression initialisieren und trainieren
model = LogisticRegression(solver='liblinear', max_iter=1000)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren (optional)
accuracy = model.score(X_test, y_test)
print("Genauigkeitswert des Modells: " + str(accuracy))

#### k-NN
- liefert durchaus gute Ergebnisse

In [ ]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

predict_data = cleaned_data.copy()

target = 'Crm.Cd'

features = ['RD', 'LONG', 'LAT', 'TIME.OCC_hour_cos']


# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()


################

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV

# Schritt 2: Daten normalisieren
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Schritt 3: Feature Selection
selector = SelectKBest(k=3)  # Wähle die 10 besten Merkmale
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# Schritt 4: Parameter Grid für die Grid Search festlegen
param_grid = {
    'n_neighbors': [3, 6, 9, 12, 15],
    'weights': ['distance', 'uniform'],
    'metric': ['manhattan', 'euclidean']
}


# Schritt 5: Grid Search durchführen
model = KNeighborsClassifier()
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train_selected, y_train)

# Schritt 6: Beste Parameter und Genauigkeit ausgeben
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_
print("Parameter: ", best_params)
print("Genauigkeit: ", best_accuracy)

# Schritt 7: Vorhersagen treffen und Genauigkeit auf Testdaten berechnen
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit auf Testdaten: ", accuracy)

#Genauigkeit mit k = 5 und keine weiteren anpassungen: Genauigkeit des k-NN-Modells (k=5): 0.36051278991467706

#######
# ['RD', 'LONG', 'LAT',
#Parameter:  {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}
#Genauigkeit:  0.20129741286336844
#Genauigkeit auf Testdaten:  0.20492888812114932

 #['RD', 'LONG', 'LAT',
#Parameter:  {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}
#Genauigkeit:  0.2084188904571302
#Genauigkeit auf Testdaten:  0.211262739678701

# ['RD', 'LONG', 'LAT', 'TIME.OCC_hour_cos']
#Parameter:  {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}
#Genauigkeit:  0.2041867226325723
#Genauigkeit auf Testdaten:  0.21273104163067888

#Parameter:  {'metric': 'manhattan', 'n_neighbors': 13, 'weights': 'distance'}
#Genauigkeit:  0.20759014324317202
#Genauigkeit auf Testdaten:  0.21584861272199327

#Parameter:  {'metric': 'manhattan', 'n_neighbors': 100, 'weights': 'distance'}
#Genauigkeit:  0.22118223326671585
#Genauigkeit auf Testdaten:  0.22800631739999505

#Parameter:  {'metric': 'manhattan', 'n_neighbors': 150, 'weights': 'distance'}
#Genauigkeit:  0.22325924510292322
#Genauigkeit auf Testdaten:  0.22984889239855555



## Finale Auswahl

- Modelle Decision Tree, Random Forest, Categorical Naive Bayes und kNN werden weiter betrachtet
- SNN und Logistic Regression haben zu lange Laufzeiten, wobei Ergebnisse nicht wirklich besser waren